## Build an Agent

This notebook is based on following langchain tutorial

- https://python.langchain.com/docs/tutorials/agents/  
- https://python.langchain.com/docs/concepts/tools/  
- https://python.langchain.com/docs/concepts/tool_calling/

### Setup

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [ ]:
import os
import getpass

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_API_KEY'] = getpass.getpass('Insert api key for LANGSMITH')

In [ ]:
os.environ['TAVILY_API_KEY'] = getpass.getpass('Insertapi key for tavily')

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

### Setup toolchain

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)

search_results = search.invoke("what is the weather in SF")
print(search_results)

In [ ]:
tools = [search]

### Setup LLM to run tools

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.language_models.chat_models import BaseChatModel, LanguageModelInput

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

llm_model: BaseChatModel = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


### Quick test with llm

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables.base import Runnable
from langchain_core.messages.base import BaseMessage, BaseMessageChunk

response: BaseMessage = llm_model.invoke([HumanMessage(content="hi!")])
response.content

In [ ]:
llm_model_with_tools: Runnable[LanguageModelInput, BaseMessage] = llm_model.bind_tools(tools)

def from_llm_tool_call(llm_model: Runnable[LanguageModelInput, BaseMessage], message: str) -> BaseMessage:
    return model_with_tools.invoke([HumanMessage(content=message)])

In [ ]:
response = from_llm_tool_call(llm_model_with_tools, 'Hi!')

print(f'Content string: {response.content}')
print(f'ToolCalls: {response.tool_calls}')

In [ ]:
response = from_llm_tool_call(llm_model_with_tools, "what's the weather in SF?")

print(f'Content string: {response.content}')
print(f'ToolCalls: {response.tool_calls}')

## Create Agent (stateless)

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import pprint

# See
# https://docs.anthropic.com/en/docs/about-claude/models/overview#model-names
model_id = "claude-3-7-sonnet-20250219"

memory = MemorySaver()
agent_model = ChatAnthropic(model_name=model_id)
agent_exc: Runnable = create_react_agent(agent_model, tools)

### Run agent

In [ ]:
def agent_call(agent: Runnable, message: str) -> BaseMessage:
    return agent_exc.invoke({"messages": [HumanMessage(content="hi")]})

In [ ]:
response = agent_call(agent_exc, "whats the weather in sf?")

In [ ]:
pprint.pprint(response['messages'])

### Streaming message

In [ ]:
from typing import Iterator

stream: Iterator[BaseMessageChunk] = agent_exc.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values")

for step in stream:
    step["messages"][-1].pretty_print()

### Streaming tokens

In [ ]:
stream: Iterator[BaseMessageChunk] = agent_exc.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages")

for step, metadata in stream:
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

## Create agent(stateful)

Pupose is to remember previous interactions

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid1

# Memory as statestore
memory = MemorySaver()

agent_exc = create_react_agent(agent_model, tools, checkpointer=memory)

In [ ]:
def do_conversation(agent: Runnable, message: str, config: dict[str, dict[str, str]]) -> Iterator[BaseMessageChunk]:
    return agent.stream(
    {"messages": [HumanMessage(content=message)]}, config
)

In [ ]:
thread_id = str(uuid1())
config = {"configurable": {"thread_id": thread_id}}

message = "hi im Bob!"
stream = do_conversation(agent_exc, message, config)

for chunk in stream:
    print(chunk)
    print('------')

In [ ]:
message = "What is my name"
stream = do_conversation(agent_exc, message, config)

for chunk in stream:
    print(chunk)
    print('------')

In [ ]:
thread_id = str(uuid1())
config = {"configurable": {"thread_id": thread_id}}

message = "What is my name"
stream = do_conversation(agent_exc, message, config)

for chunk in stream:
    print(chunk)
    print('------')